In [1]:
#function to test models 
import nest_asyncio
nest_asyncio.apply()

In [2]:
from mb_pytorch.models.modelloader import ModelLoader
from mb_pytorch.dataloader.loader import DataLoader

In [3]:
k = DataLoader('../scripts/model_load/loader_y.yaml')

In [4]:
model_data_dict = k.data_dict['model']

In [5]:
model_data_load = ModelLoader(k.data_dict['model'])

model_data =  model_data_load.get_model()

train_loader,val_loader,_,_ = k.data_load()

all_data = k.data_dict

/home/malav/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/malav/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
from mb_pytorch.training.train_params import train_helper

model_yaml_data = k.data_dict['model']
loss_attr,optimizer_attr,optimizer_dict,scheduler_attr,scheduler_dict = train_helper(model_yaml_data) 

In [7]:
print(loss_attr())
print(optimizer_attr)
print(optimizer_dict)
print(scheduler_attr)
print(scheduler_dict)

CrossEntropyLoss()
<class 'torch.optim.adam.Adam'>
{'lr': 0.001, 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}
<class 'torch.optim.lr_scheduler.StepLR'>
{'step_size': 3, 'gamma': 0.2}


In [8]:

from torch.utils.tensorboard import SummaryWriter
## setting the trianing params

optimizer =optimizer_attr(model_data.parameters(),**optimizer_dict)
scheduler = scheduler_attr(optimizer,**scheduler_dict)

best_val_loss = float('inf')

path_logs = os.path.join(all_data['data']['work_dir'], 'logs')
writer = SummaryWriter(log_dir=path_logs)

all_model_data= k.data_dict['model']


In [9]:
import torch
from mb_utils.src.logging import logger
from itertools import islice

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader10 = islice(train_loader, 50)
val_loader10 = islice(val_loader, 50)


In [10]:
from mb_pytorch.classification.training import train_loop

train_loop(k,model_data_dict,model_data,train_loader,val_loader,loss_attr,optimizer,scheduler,writer,logger,gradcam=['avgpool'],device=device)

  0%|          | 0/50 [00:00<?, ?it/s]Wed 10:05:24,797 INF Training Started                                                                                                
Wed 10:05:27,700 INF Epoch 1 - Batch 1 - Train Loss: 2.686540126800537                                                               
Wed 10:05:29,851 INF Epoch 1 - Batch 2 - Train Loss: 3.1688127517700195                                                              
Wed 10:05:31,784 INF Epoch 1 - Batch 3 - Train Loss: 3.289283037185669                                                               
Wed 10:05:33,684 INF Epoch 1 - Batch 4 - Train Loss: 3.277287483215332                                                               
Wed 10:05:35,641 INF Epoch 1 - Batch 5 - Train Loss: 2.684417963027954                                                               
Wed 10:05:37,541 INF Epoch 1 - Batch 6 - Train Loss: 2.85557222366333                                                                
Wed 10:05:39,499 INF Epo

AttributeError: 'str' object has no attribute 'register_forward_hook'

In [34]:
import tqdm
from torch.utils.tensorboard import SummaryWriter
import os


for i in tqdm.tqdm(range(all_model_data['model_epochs'])):
    ##train loop
    model_data.train()
    train_loss = 0
    for j,(x,y) in enumerate(train_loader10):
        x,y = x.to(device),y.to(device)
        optimizer.zero_grad()
        y_pred = model_data(x)
        current_loss = loss_attr()(y_pred,y)
        current_loss.backward()    
        optimizer.step()
        train_loss += current_loss.item()
        logger.info(f'Epoch {i+1} - Batch {j+1} - Train Loss: {current_loss.item()}')
    
    avg_train_loss = train_loss / len(train_loader)
    logger.info(f'Epoch {i+1} - Train Loss: {avg_train_loss}')
    
    writer.add_scalar('Loss/train', avg_train_loss, global_step=i)
        
    if scheduler is not None:
        scheduler.step()
    
    for name, param in model_data.named_parameters():
        writer.add_histogram(name, param, global_step=i)
        
    #get grad cam images
        
    #validation loop
    val_loss = 0
    val_acc = 0
    num_samples = 0
    
    model_data.eval()
    with torch.no_grad():
        for x_val, y_val in val_loader10:
            x_val, y_val = x_val.to(device), y_val.to(device)
            output = model_data(x_val)
            val_loss += loss_attr()(output, y_val).item() * x_val.size(0)
            _, preds = torch.max(output, 1)
            val_acc += torch.sum(preds == y_val.data)
            num_samples += x_val.size(0)
            logger.info(f'Epoch {i+1} - Batch {j+1} - Val Loss: {val_loss}')
        
        if num_samples == 0:
            num_samples = 1
        val_loss /= num_samples
        val_acc = val_acc / num_samples
        logger.info(f'Epoch {i+1} - Val Loss: {val_loss}', f'Epoch {i+1} - Val Acc: {val_acc}')
    
    writer.add_scalar('Loss/val', val_loss, global_step=i)
    writer.add_scalar('Accuracy/val', val_acc, global_step=i)
    
    # save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model_data.state_dict()

        path = os.path.join(all_data['data']['work_dir'], 'best_model.pth')
        torch.save(best_model, path)
        logger.info(f'Epoch {i+1} - Best Model Saved')
        
    model_data.train()
        


  0%|          | 0/50 [00:00<?, ?it/s]Tue 01:34:05,255 INF Epoch 1 - Batch 1 - Train Loss: 2.7716147899627686                                                              
Tue 01:34:07,058 INF Epoch 1 - Batch 2 - Train Loss: 3.0471785068511963                                                              
Tue 01:34:08,862 INF Epoch 1 - Batch 3 - Train Loss: 3.576723575592041                                                               
Tue 01:34:10,702 INF Epoch 1 - Batch 4 - Train Loss: 3.355401039123535                                                               
Tue 01:34:12,620 INF Epoch 1 - Batch 5 - Train Loss: 3.2821779251098633                                                              
Tue 01:34:14,500 INF Epoch 1 - Batch 6 - Train Loss: 3.361398220062256                                                               
Tue 01:34:16,429 INF Epoch 1 - Batch 7 - Train Loss: 3.22971510887146                                                                
Tue 01:34:18,368 INF Epo

In [84]:
from mb_pytorch.utils.extra_utils import feature_view

for l in train_loader10:
    x,y = l
    break

from mb_pytorch.utils.extra_utils import get_model_summary
get_model_summary(model_data,input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,096
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          16,384
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13            [-1, 256, 8, 8]          16,384
      BatchNorm2d-14            [-1, 25

In [107]:
list(model_data.named_modules()).index(('conv1', model_data.conv1))

1

In [114]:
model_data.layer4[-1]

Bottleneck(
  (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
)

In [126]:
list(model_data.named_modules()).index(('fc', model_data.fc))

150

In [119]:
feature_view(x[0],model_data,layer_names=['layer4.relu'])

ValueError: 'model.layer4.relu' is not in list